### 0. Импорт библиотек

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# для Tfidf
!pip install pymorphy3
!

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 78.3 MB/s eta 0:00:00


In [22]:
# база
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
from tqdm import tqdm
from time import time
# ML
import pymorphy3  #для работы с русским текстом
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix,  classification_report
# DL
import torch
from torch import nn
# импортируем трансформеры
import transformers
import warnings
warnings.filterwarnings('ignore')

### 1. Импорт датасета

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [5]:
data_path = '/content/drive/MyDrive/Colab_Notebooks/10-week_NLP/data'

In [6]:
df = pd.read_json(f'{data_path}/kinopoisk.jsonl', lines=True)
df.head()

,part,movie_name,review_id,author,date,title,grade3,grade10,content
0,top250,Блеф (1976),17144,Come Back,2011-09-24,Плакали наши денежки ©,Good,10.0,"\n""Блеф» — одна из моих самых любимых комедий...."
1,top250,Блеф (1976),17139,Stasiki,2008-03-04,None,Good,0.0,\nАдриано Челентано продолжает радовать нас св...
2,top250,Блеф (1976),17137,Flashman,2007-03-04,None,Good,10.0,"\nНесомненно, это один из великих фильмов 80-х..."
3,top250,Блеф (1976),17135,Sergio Tishin,2009-08-17,""" Черное, красное, ерунда это все. Выигрывает ...",Good,0.0,\nЭта фраза на мой взгляд отражает сюжет несом...
4,top250,Блеф (1976),17151,Фюльгья,2009-08-20,"«Он хотел убежать? Да! Блеф, блеф…»",Neutral,7.0,"\n- как пела Земфира, скорее всего, по соверше..."


In [7]:
# проверка на сбалансированность
df['grade3'].value_counts(normalize=True)*100

,proportion
grade3,
Good,74.510125
Bad,12.984067
Neutral,12.505807


In [8]:
reviews_df = df[['grade3', 'content']]

In [9]:
reviews_df['content'][0]

'\n"Блеф»\xa0—\xa0одна из\xa0моих самых любимых комедий.\n\nЭтот фильм я\xa0наверно смотрел раз\xa0сто, нет\xa0я конечно блефую, я\xa0видел его\xa0куда больше. Не\xa0могу не\xa0выразить своё восхищение главными действующими лицами этого фильма. Начну с\xa0Адриано Челентано для\xa0которого как\xa0я считаю это\xa0лучшая роль в\xa0кино. Великолепный актёр, неплохой певец, странно что\xa0на его\xa0родине в\xa0Италии его\xa0песни мало кто\xa0слушает. Ну\xa0я думаю что\xa0и итальянцы и\xa0французы привыкли к\xa0тому, что\xa0у нас\xa0до сих\xa0их актёры популярней чем\xa0даже на\xa0своей родине. Да, такой вот\xa0парадокс. Челентано конечно профессионал своего дела, комик с\xa0серьёзным выражением лица. Он\xa0смешон ещё\xa0и потому, что\xa0одновременно так\xa0серъёзен. Адриано браво!\n\nА\xa0теперь несколько слов об\xa0Энтони Куине. Да\xa0тот самый горбун из\xa0Нотр-дама. Собор Парижской Богоматери, оригинальная версия, кто\xa0не смотрел рекомендую. С\xa0ним как-то приключилась одна интересная

In [10]:
# ф-я для очистки текста
def clean_text(text):
    all_punct = string.punctuation + '"'
    # удаляем ссылки
    text = re.sub(r'http\S+', " ", text)
    # Сначала разделяем слова по регистру (camelCase и PascalCase)
    text = re.sub(r'([а-яa-z])([А-ЯA-Z])', r'\1 \2', text)
    # перводим в нижний регистр
    text = text.lower()
    # удаляем числа
    text = re.sub(r'\d+', ' ', text)
    # удаляем emoji
    text = re.sub(r'[^\w\s,\.!?;:]', '', text)
    # Удаляем знаки пунктуации, но сохраняем точку, восклицательный и вопросительный знаки
    # для последующей обработки
    punct_to_remove = all_punct.replace('.', '').replace('!', '').replace('?', '')
    text = text.translate(str.maketrans('', '', punct_to_remove))
    # Добавляем пробелы после точек, восклицательных и вопросительных знаков, если их нет
    # Обрабатываем случаи типа "слово.Следующее" -> "слово. Следующее"
    text = re.sub(r'([.!?])([а-яa-z])', r'\1 \2', text)
    # Удаляем оставшиеся знаки пунктуации
    text = text.translate(str.maketrans('', '', all_punct))
     # Заменяем все последовательности пробелов на один пробел
    text = re.sub(r'\s+', ' ', text).strip()
    return text


In [11]:
clean_text(reviews_df['content'][0])

'блеф одна из моих самых любимых комедий этот фильм я наверно смотрел раз сто нет я конечно блефую я видел его куда больше не могу не выразить своё восхищение главными действующими лицами этого фильма начну с адриано челентано для которого как я считаю это лучшая роль в кино великолепный актёр неплохой певец странно что на его родине в италии его песни мало кто слушает ну я думаю что и итальянцы и французы привыкли к тому что у нас до сих их актёры популярней чем даже на своей родине да такой вот парадокс челентано конечно профессионал своего дела комик с серьёзным выражением лица он смешон ещё и потому что одновременно так серъёзен адриано браво а теперь несколько слов об энтони куине да тот самый горбун из нотрдама собор парижской богоматери оригинальная версия кто не смотрел рекомендую с ним както приключилась одна интересная история на съёмках одного из своих фильмов он то ли сломал то ли подвихнул ногу а роль требовала от него чтобы в одной из сцен он кружился с дамой в танце и он

### 2. ML: TF-IDF

In [12]:
# очищенный текст
cleaned_text = []
for text in reviews_df['content']:
  cleaned_text.append(clean_text(text))

In [15]:
# лемматизация
from functools import lru_cache

morph = pymorphy3.MorphAnalyzer()
# Создаем кэшированную версию функции
@lru_cache(maxsize=100000)
def lemmitize_word_cached(word):
    return morph.parse(word)[0].normal_form

def lemmatize_texts_optimized(texts):
    lemmatized_texts = []
    for tweet in texts:
        lemmatized_texts.append(' '.join([lemmitize_word_cached(word) for word in tweet.split()]))
    return lemmatized_texts

In [16]:
lemmatized_text = lemmatize_texts_optimized(cleaned_text)

In [17]:
# Сохраняем в CSV
reviews_df['lemmatized_text'] = lemmatized_text
reviews_df
#df.to_csv(f'{data_path}/lemmatized_kinopoisk.csv', index=False, encoding='utf-8-sig')

,grade3,content,lemmatized_text
0,Good,"\n""Блеф» — одна из моих самых любимых комедий....",блеф один из мой самый любимый комедия этот фи...
1,Good,\nАдриано Челентано продолжает радовать нас св...,адриано челентано продолжать радовать мы свой ...
2,Good,"\nНесомненно, это один из великих фильмов 80-х...",несомненно это один из великий фильм х год иск...
3,Good,\nЭта фраза на мой взгляд отражает сюжет несом...,этот фраза на мой взгляд отражать сюжет несомн...
4,Neutral,"\n- как пела Земфира, скорее всего, по соверше...",как петь земфира скорее весь по совершенно дру...
...,...,...,...
36586,Bad,\n Ну с чего бы начать… Давненько я не пи...,ну с что бы начать давненько я не писать отриц...
36587,Bad,"\n Можно начать с того, что уже постер к ...",можно начать с тот что уже постер к фильм цвет...
36588,Good,"\n Фильм производства России, поэтому мно...",фильм производство россия поэтому многие ожида...
36589,Bad,\n 16 сентября на большие экраны вышел «м...,сентябрь на больший экран выйти мистический тр...


In [18]:
# токенизация
reg_tokenizer = RegexpTokenizer(r'\w+')

tokenized_text = reg_tokenizer.tokenize_sents(lemmatized_text)

In [20]:
# Скачиваем стоп-слова
try:
    stop_words_ru = set(stopwords.words('russian'))
    stop_words_en = set(stopwords.words('english'))
except:
    nltk.download('stopwords')
    nltk.download('punkt')
    stop_words_ru = set(stopwords.words('russian'))
    stop_words_en = set(stopwords.words('english'))

# Объединяем и добавляем дополнительные стоп-слова
add_russian = pd.read_json(f'{data_path}/stopwords-ru.json')[0].tolist()

stop_words_combined = stop_words_ru.union(stop_words_en).union(add_russian)

# Создаем кэшированную функцию для проверки стоп-слов
@lru_cache(maxsize=100000)
def is_stopword(word):
    """Проверяет, является ли слово стоп-словом"""
    return word.lower() in stop_words_combined

def remove_stopwords(text):
    """Удаляет стоп-слова из текста"""
    clean_text = []

    for element in tqdm(text, desc="Удаление стоп-слов", unit="текст"):
        # Фильтруем слова, оставляя только не стоп-слова
        filtered_words = [word for word in element if not is_stopword(word)]
        clean_text.append(' '.join(filtered_words))

    return clean_text

clean_tokenized_text = remove_stopwords(tokenized_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Удаление стоп-слов: 100%|██████████| 36591/36591 [00:03<00:00, 11477.94текст/s]


In [23]:
# векторизация
vectorizer = TfidfVectorizer(
    max_df=0.5,
    min_df=5,
    stop_words=list(stop_words_combined)
)
t0 = time()
X_tfidf = vectorizer.fit_transform(clean_tokenized_text)

print(f"vectorization done in {time() - t0:.3f} s")
print(f"n_samples: {X_tfidf.shape[0]}, n_features: {X_tfidf.shape[1]}")

vectorization done in 6.077 s
n_samples: 36591, n_features: 32400


In [ ]:
# Кодирование меток (строки) в числа для обучения
le = LabelEncoder()
y_encoded = le.fit_transform(reviews_df['grade3'])

X_train, X_test, y_train, y_test = train_test_split(
   X_tfidf, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

In [ ]:
# Обучение Логистической регрессии
model_LogReg_tfidf = LogisticRegression(class_weight='balanced')
model_LogReg_tfidf.fit(X_train, y_train)
y_pred = model_LogReg_tfidf.predict(X_test)
f1_macro = f1_score(y_test, y_pred, average='macro')
# y_pred_labels = le.inverse_transform(y_pred)

In [ ]:
# Обчение Random Forest
model_RanFor_tfidf = RandomForestClassifier(class_weight='balanced')
model_RanFor_tfidf.fit(X_train, y_train)
y_pred = model_RanFor_tfidf.predict(X_test)
f1_macro = f1_score(y_test, y_pred, average='macro')
# y_pred_labels = le.inverse_transform(y_pred)

In [ ]:
# сохранение лучшей модели
import joblib
save_path = '/content/drive/MyDrive/Colab_Notebooks/10-week_NLP/models/film_review'
# Сохранение
joblib.dump(clf, f'{save_path}/model_tfidf.joblib')
# Загрузка
#clf = joblib.load('model.joblib')

### 3. LSTM

### 3. BERT-based

#### 3.1 Импорт rubert-tiny2

In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer_bert = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model_bert = AutoModel.from_pretrained("cointegrated/rubert-tiny2")

#### 3.2 Encoding

In [ ]:
# Выбираем подходящий max_length
lengths = reviews_df['content'].apply(lambda x: len(tokenizer_bert.tokenize(x)))
print(lengths.describe())  # статистика, включая перцентили

In [ ]:
post_max_length =
BATCH_SIZE = 64

In [ ]:
# Эмбеддинг (получение векторного представления) текста
encoded_posts = reviews_df['content'].apply(
    lambda x: tokenizer_bert(x, padding='max_length', max_length=post_max_length, truncation=True, return_tensors='pt')
).values

In [ ]:
# Класс создаёт адаптер (dataset) для подготовки батчей данных из закодированных постов (BERT inputs) для подачи в DataLoader PyTorch.
# Он превращает закодированные посты (получаемые токенизатором) в объекты, которые удобно брать по индексу — например, для обучения или инференса модели.
class BertInputs(torch.utils.data.Dataset):
    def __init__(self, encoded_posts):
        super().__init__()
        self.inputs = encoded_posts

    def __len__(self):
        return self.inputs.shape[0]

    def __getitem__(self, idx):
        # print(self.inputs[idx])
        # лишнее измерение приходит из-за индивидуальной токенизации с добавленным размером батча для одного текста
        # его нужно убрать в Dataset или делать токенизацию пакетно для всей коллекции сразу
        return (torch.Tensor(self.inputs[idx]['input_ids']).squeeze(0).to(device).long(),
                torch.Tensor(self.inputs[idx]['attention_mask']).squeeze(0).to(device).long())

In [ ]:
# Создание датасета по кастомному BertInputs (класс)
dataset = BertInputs(encoded_posts)
# Создание лоудера
loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False)
batch = next(iter(loader))
print(f'{len(batch[0])} x {len(batch[0][0])}') # shape BATCH_SIZE x post_max_length

In [ ]:
%%time
model_bert.to(device)

features = [] # Список для аккумулирования эмбеддингов
# Итерация по батчам из DataLoader
for inputs, masks in loader:
    with torch.inference_mode():
        model_out = model_bert(inputs, attention_mask=masks)
        vectors = model_out.last_hidden_state[:, 0, :]
    features.extend(vectors.cpu().numpy())
len(features)

In [ ]:
model_out_shape = model_out.last_hidden_state.shape
print(model_out_shape)
print(f'{model_out_shape[0]} - число объектов,\n\
{model_out_shape[1]} - длина каждой последовательности, \n\
{model_out_shape[2]} - длина выходного вектора BERT для одного элемента последовательности')

#### 3.3 traintest, обучение

In [ ]:
le = LabelEncoder()
y_encoded = le.fit_transform(reviews_df['grade3'])

X_train_bert, X_val_bert, y_train_bert, y_val_bert = train_test_split(features, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded)

In [19]:
# Обучение Логистической регрессии
model_LogReg_bert = LogisticRegression(class_weight='balanced')
model_LogReg_bert.fit(X_train_bert, y_train_bert)
y_pred_bert = model_LogReg_bert.predict(X_val_bert)
f1_macro = f1_score(y_val_bert, y_pred_bert, average='macro')
# y_pred_labels = le.inverse_transform(y_pred_bert)

NameError: name 'LogisticRegression' is not defined

In [ ]:
# Обучение Random Forest
model_RanFor_bert = RandomForestClassifier(class_weight='balanced')
model_RanFor_bert.fit(X_train_bert, y_train_bert)
y_pred_bert = model_RanFor_bert.predict(X_val_bert)
f1_macro = f1_score(y_val_bert, y_pred_bert, average='macro')
# y_pred_labels = le.inverse_transform(y_pred_bert)

In [ ]:
# сохранение лучшей модели
save_path = '/content/drive/MyDrive/Colab_Notebooks/10-week_NLP/models/film_review'
# Сохранение
joblib.dump(clf, f'{save_path}/model_bert.joblib')
# Загрузка
#clf = joblib.load('model.joblib')